In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdmolops import GetFormalCharge
from rdkit.Chem.rdMolDescriptors import CalcMolFormula, CalcExactMolWt
from pandas import Series,DataFrame
import pandas as pd

In [3]:
D = pd.read_csv('data.csv')
adduct_type = list(D['Adduct'])
pubchem_id  = list(D['Pubchem ID'])
exact_min   = list(D['Exac Min'])
exact_max   = list(D['Exac Max'])
ccs_min     = list(D['CCS Min'])
ccs_max     = list(D['CCS Max'])
ccs_true    = list(D['CCS'])
exact_true  = list(D['Exac'])
ID          = list(D['ID'])
D

,Pubchem ID,Inchi,Exac Min,Exac Max,CCS Min,CCS Max,Exac,CCS,Adduct,ID
0,5280637,InChI=1S/C21H20O11/c22-7-16-18(27)19(28)20(29)...,448.088695,448.111100,201.21,216.81,448.099898,209.006823,[M+H]+,Y1
1,442439,InChI=1S/C28H34O15/c1-10-21(33)23(35)25(37)27(...,610.173852,610.204362,220.40,238.60,610.189107,229.497183,[M-H]-,Y2
2,30231,InChI=1S/C28H36O15/c1-11-21(34)23(36)25(38)27(...,612.189452,612.220062,225.22,242.69,612.204757,233.956908,[M+H]+,Y3
3,73659,InChI=1S/C30H48O4/c1-25(2)12-14-30(24(33)34)15...,472.342788,472.366405,213.71,231.36,472.354597,222.535275,[M-H]-,Y4
4,9809542,"InChI=1S/C36H62O9/c1-19(2)10-9-13-36(8,45-31-2...",638.422759,638.454681,248.90,269.46,638.438720,259.180746,[M-H]-,Y5


In [1]:
fenshu = []
excat_num = []
ccs_num = []
for index in range(len(adduct_type)):
    
    data_exm = []
    data_ccs = []

    for i in range(1,156000000,500000):
        filename = str(i).zfill(9) + '_' + str(i+500000-1).zfill(9)
        
        data = pd.read_csv('Compound_'+filename+'.csv')
        data = data[data['EXACT MASS'] >= exact_min[index]]
        data = data[data['EXACT MASS'] <= exact_max[index]]
        data_exm.append(data)
        
        data = data[data[adduct_type[index]] >= ccs_min[index]]
        data = data[data[adduct_type[index]] <= ccs_max[index]]
        data_ccs.append(data)
        
    data_exm_s = pd.concat(data_exm, axis=0, ignore_index=True)
    data_ccs_s = pd.concat(data_ccs, axis=0, ignore_index=True)
    
    excat_num.append(len(list(data_exm_s['Pubchem ID'])))
    ccs_num.append(len(list(data_ccs_s['Pubchem ID'])))
    
    inchis = []
    data_ccs_s_smiles = list(data_ccs_s['ISO SMILES'])
    for smi in data_ccs_s_smiles:
        mol = Chem.MolFromSmiles(smi)
        inchi = Chem.inchi.MolToInchi(mol)
        inchis.append(inchi)
    data_ccs_s['INCHI'] = inchis
    
    RE = []
    ccs_pred = list(data_ccs_s[adduct_type[index]])
    for ccs in ccs_pred:
        RE.append((ccs_true[index]-ccs)/ccs_true[index]*100)
        
    try:
        this_ccs_pred = list(data_ccs_s[data_ccs_s['Pubchem ID'].isin([pubchem_id[index]])][adduct_type[index]])[0]
        this_ccs_re = (ccs_true[index]-this_ccs_pred)/ccs_true[index]*100
        if this_ccs_re < 0:
            fenshu.append((1-this_ccs_re/min(RE))*100)
        else:
            fenshu.append((1-this_ccs_re/max(RE))*100)
    except:
        fenshu.append('null')
    
    data_ccs_s.to_csv(str(ID[index])+'.csv',index=False)

In [5]:
D['excat_num'] = excat_num
D['ccs_num'] = ccs_num
D

,Pubchem ID,Inchi,Exac Min,Exac Max,CCS Min,CCS Max,Exac,CCS,Adduct,ID,excat_num,ccs_num
0,5280637,InChI=1S/C21H20O11/c22-7-16-18(27)19(28)20(29)...,448.088695,448.111100,201.21,216.81,448.099898,209.006823,[M+H]+,Y1,14629,9472
1,442439,InChI=1S/C28H34O15/c1-10-21(33)23(35)25(37)27(...,610.173852,610.204362,220.40,238.60,610.189107,229.497183,[M-H]-,Y2,3827,1099
2,30231,InChI=1S/C28H36O15/c1-11-21(34)23(36)25(38)27(...,612.189452,612.220062,225.22,242.69,612.204757,233.956908,[M+H]+,Y3,3944,1542
3,73659,InChI=1S/C30H48O4/c1-25(2)12-14-30(24(33)34)15...,472.342788,472.366405,213.71,231.36,472.354597,222.535275,[M-H]-,Y4,2254,1807
4,9809542,"InChI=1S/C36H62O9/c1-19(2)10-9-13-36(8,45-31-2...",638.422759,638.454681,248.90,269.46,638.438720,259.180746,[M-H]-,Y5,733,505
